In [1]:
import pandas as pd
import re
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from statistics import mean, stdev
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import model_selection

# Prepare Dataset

In [2]:
# import dataset
df = pd.read_csv('DATA/train_preprocess.tsv.txt', sep='\t', names=['Tweet','Label'])
df.head()

,Tweet,Label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [3]:
df.Label.value_counts()

Label
positive    6416
negative    3436
neutral     1148
Name: count, dtype: int64

In [4]:
# Menghapus duplikat pada kolom "tweet"
df = df.drop_duplicates(subset='Tweet')

# Memeriksa data setelah menghapus duplikat
print("\nData setelah menghapus duplikat:")
print(df)


Data setelah menghapus duplikat:
                                                   Tweet     Label
0      warung ini dimiliki oleh pengusaha pabrik tahu...  positive
1      mohon ulama lurus dan k212 mmbri hujjah partai...   neutral
2      lokasi strategis di jalan sumatera bandung . t...  positive
3      betapa bahagia nya diri ini saat unboxing pake...  positive
4      duh . jadi mahasiswa jangan sombong dong . kas...  negative
...                                                  ...       ...
10993  f - demokrat dorong upaya kemandirian energi n...   neutral
10994                                        tidak bosan  positive
10996  enak rasa masakan nya apalagi kepiting yang me...  positive
10998  pagi pagi di tol pasteur sudah macet parah , b...  negative
10999  meskipun sering belanja ke yogya di riau junct...  positive

[10933 rows x 2 columns]


# Text Normalization

In [5]:
def cleansing(sent):
    # Mengubah kata menjadi huruf kecil semua dengan menggunakan fungsi lower()
    string = sent.lower()

    # Menghapus emoticon dan tanda baca menggunakan "RegEx" dengan script di bawah
    string = re.sub(r'(?:\@|http?\://|https?\://|www)\S+', '', string) #menghapus https dan http
    string = re.sub('<.*?>', ' ', string) #mengganti karakter html dengan tanda petik
    string = re.sub('[^0-9a-zA-Z]+', ' ', string) #menghilangkan semua karakter yang bukan huruf atau angka dan menggantinya dengan spasi.
    string = re.sub('\n',' ',string) #mengganti line baru dengan spasi
    string = re.sub(r':', ' ', string) #menggantikan karakter : dengan spasi 
    string = re.sub('gue','saya', string) # Mengganti kata "gue" dengan kata "saya"
    string = re.sub(r'\b[a-zA-Z]\b', ' ', string) #menghapus single char
    string = ' '.join(string.split()) #memisahkan dan menggabungkan kata
    string = string.strip() #menghilangkan whitespace di awal dan di akhir teks
    string = re.sub(r'pic.twitter.com.[\w]+', '', string) #menghapus link picture
    string = re.sub(r'\buser\b',' ', string) #menghapus kata 'user'
    string = re.sub(r'\brt\b',' ', string) #menghapus awalan rt
    string = re.sub('RT',' ', string) #menghapus RT simbol
    string = re.sub(r'‚Ä¶', '', string) #menghapus simbol tidak perlu

    # Lematisasi menggunakan Sastrawi
    stemmer_factory = StemmerFactory()
    stemmer = stemmer_factory.create_stemmer()
    string = stemmer.stem(string)

    # Menghapus stop words menggunakan Sastrawi
    stopword_factory = StopWordRemoverFactory()
    stopword_remover = stopword_factory.create_stop_word_remover()
    string = stopword_remover.remove(string)
    
    return string

In [6]:
df['Tweet_Clean'] = df.Tweet.apply(cleansing)
df.head()

,Tweet,Label,Tweet_Clean
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung milik usaha pabrik puluh tahun kenal pu...
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral,ulama lurus k212 mmbri hujjah partai diwlh sua...
2,lokasi strategis di jalan sumatera bandung . t...,positive,lokasi strategis jalan sumatera bandung nyaman...
3,betapa bahagia nya diri ini saat unboxing pake...,positive,betapa bahagia unboxing paket barang bagus beli
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative,duh mahasiswa sombong kasih kartu kuning ajar ...


# Feature Extraction

## TF-IDF

In [7]:
data_preprocessed = df.Tweet_Clean.tolist()

In [8]:
# Proses feature extraction
tfidf_vect = TfidfVectorizer()

X = tfidf_vect.fit_transform(data_preprocessed)
print("Feature extraction selesai")

Feature extraction selesai


In [9]:
pickle.dump(tfidf_vect, open("feature.p", "wb"))

In [10]:
tfidf_array = X.toarray()
df_array = pd.DataFrame(data=tfidf_array, columns=tfidf_vect.get_feature_names_out())
print(df_array)

        00  000  001   01  010  0111  011770465655617   02  021  022  ...  \
0      0.0  0.0  0.0  0.0  0.0   0.0              0.0  0.0  0.0  0.0  ...   
1      0.0  0.0  0.0  0.0  0.0   0.0              0.0  0.0  0.0  0.0  ...   
2      0.0  0.0  0.0  0.0  0.0   0.0              0.0  0.0  0.0  0.0  ...   
3      0.0  0.0  0.0  0.0  0.0   0.0              0.0  0.0  0.0  0.0  ...   
4      0.0  0.0  0.0  0.0  0.0   0.0              0.0  0.0  0.0  0.0  ...   
...    ...  ...  ...  ...  ...   ...              ...  ...  ...  ...  ...   
10928  0.0  0.0  0.0  0.0  0.0   0.0              0.0  0.0  0.0  0.0  ...   
10929  0.0  0.0  0.0  0.0  0.0   0.0              0.0  0.0  0.0  0.0  ...   
10930  0.0  0.0  0.0  0.0  0.0   0.0              0.0  0.0  0.0  0.0  ...   
10931  0.0  0.0  0.0  0.0  0.0   0.0              0.0  0.0  0.0  0.0  ...   
10932  0.0  0.0  0.0  0.0  0.0   0.0              0.0  0.0  0.0  0.0  ...   

       zonpoliticon  zoo  zoom  zubir  zulfikri  zulkarnain  zup  zupa  zup

## Prepare Train and Test Dataset (Split Dataset)

In [11]:
classes = df.Label

# split dataset menjadi 80% untuk train dan 20% untuk test
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size=0.2)

In [12]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8746, 12862)
(2187, 12862)
(8746,)
(2187,)


# Model Training

## Neural Network

In [13]:
# model training menggunakan neural network
model = MLPClassifier()
model.fit(X_train, y_train)

print("Training selesai")

Training selesai


In [14]:
pickle.dump(model, open("model.p", "wb"))

# Model Evaluation

In [15]:
test = model.predict(X_test)

print("Testing selesai")
print(classification_report(y_test, test))

Testing selesai
              precision    recall  f1-score   support

    negative       0.68      0.73      0.70       653
     neutral       0.68      0.49      0.57       227
    positive       0.86      0.87      0.86      1307

    accuracy                           0.79      2187
   macro avg       0.74      0.70      0.71      2187
weighted avg       0.79      0.79      0.79      2187



## Cross Validation

In [16]:
X = df[["Tweet_Clean"]] #input feature
Y = df["Label"] #output feature

# split dataset menjadi 80% untuk train dan 20% untuk test
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size=0.2)

In [17]:
# cross validation dengan split = 5
kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []

X_train = X_train.reset_index().Tweet_Clean
y_train = y_train.reset_index().Label

def cross_validation(k, X_train, model, name):
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    # iterate sebanyak k_cross validation
    for iteration, data in enumerate(kf.split(X_train), start=1):
        # mengambil data dan target train
        data_train = X_train[data[0]]
        tfidf_vect = TfidfVectorizer()
        data_train = tfidf_vect.fit_transform(data_train)
        target_train = y_train[data[0]]

        # mengambil data dan target test
        data_test = y_train[data[1]]
        data_test = tfidf_vect.transform(data_test)
        target_test = y_train[data[1]]

        # model training menggunakan data train
        classifier = model
        classifier.fit(data_train, target_train)

        # prediksi data test
        preds = classifier.predict(data_test)

        # menghitung accuracy
        accuracy = accuracy_score(target_test, preds)
        precision = precision_score(target_test, preds, average='weighted')
        recall = recall_score(target_test, preds, average='weighted')
        f1 = f1_score(target_test, preds, average='weighted')

        #print("Training ke-", iteration)
        #print(classification_report(target_test, preds))
        #print("=================================================================")

        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    result = {'algorithm': name,
                'accuracy_per_iter': accuracies, 'accuracy_mean': mean(accuracies), 'accuracy_std': stdev(accuracies),
                'recall_mean': mean(recalls),'precision_mean': mean(precisions), 'f1_mean': mean(f1_scores)}
    return result

In [18]:
result = cross_validation(5, X_train, MLPClassifier(), "Neural Network")
result

C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

{'algorithm': 'Neural Network',
 'accuracy_per_iter': [0.30514285714285716,
  0.3047455688965123,
  0.3270440251572327,
  0.5603201829616924,
  0.3098913664951401],
 'accuracy_mean': 0.3614288001306869,
 'accuracy_std': 0.11155413266808988,
 'recall_mean': 0.3614288001306869,
 'precision_mean': 0.14058623717617186,
 'f1_mean': 0.19905891725525748}

## Compare All Models with Cross Validation

In [19]:
X = df[["Tweet_Clean"]] #input feature
Y = df["Label"] #output feature

# split dataset menjadi 80% untuk train dan 20% untuk test
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size=0.2)

X_train = X_train.reset_index().Tweet_Clean
y_train = y_train.reset_index().Label

X_test = X_test.reset_index().Tweet_Clean
y_test = y_test.reset_index().Label

In [20]:
# loading model
models = []
models.append(('Logistic Regression', LogisticRegression()))
models.append(('K Nearest Neighbour', KNeighborsClassifier()))
models.append(('Naive Bayes', MultinomialNB()))
models.append(('Support Vector Machine', SVC()))
models.append(('Neural Network', MLPClassifier()))
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('Random Forest', RandomForestClassifier()))
models.append(('Gradient Boosting', GradientBoostingClassifier()))
models.append(('AdaBoost', AdaBoostClassifier()))

In [21]:
# evaluating the model
results = []
for name, model in models:
      print(name)
      result = cross_validation(5, X_train, model, name)
      results.append(result)
      print(result)
results = pd.DataFrame(results)
results = results.sort_values("accuracy_mean",ascending=False)

Logistic Regression


C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

{'algorithm': 'Logistic Regression', 'accuracy_per_iter': [0.3068571428571429, 0.32075471698113206, 0.29559748427672955, 0.3270440251572327, 0.31503716409376786], 'accuracy_mean': 0.313058106673201, 'accuracy_std': 0.012268717140233663, 'recall_mean': 0.313058106673201, 'precision_mean': 0.09812579529002295, 'f1_mean': 0.14938476549249913}
K Nearest Neighbour


C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

{'algorithm': 'K Nearest Neighbour', 'accuracy_per_iter': [0.3068571428571429, 0.32075471698113206, 0.6049170954831332, 0.5637507146941109, 0.31503716409376786], 'accuracy_mean': 0.4222633668218574, 'accuracy_std': 0.1487456850429259, 'recall_mean': 0.4222633668218574, 'precision_mean': 0.19600657401484195, 'f1_mean': 0.2626649252778666}
Naive Bayes


...

C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

{'algorithm': 'Gradient Boosting', 'accuracy_per_iter': [0.3068571428571429, 0.32075471698113206, 0.29559748427672955, 0.3270440251572327, 0.31503716409376786], 'accuracy_mean': 0.313058106673201, 'accuracy_std': 0.012268717140233663, 'recall_mean': 0.313058106673201, 'precision_mean': 0.09812579529002295, 'f1_mean': 0.14938476549249913}
AdaBoost


C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

{'algorithm': 'AdaBoost', 'accuracy_per_iter': [0.3068571428571429, 0.32075471698113206, 0.29559748427672955, 0.3270440251572327, 0.31503716409376786], 'accuracy_mean': 0.313058106673201, 'accuracy_std': 0.012268717140233663, 'recall_mean': 0.313058106673201, 'precision_mean': 0.09812579529002295, 'f1_mean': 0.14938476549249913}


C:\Users\Reza Fakhrurrozi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
results

,algorithm,accuracy_per_iter,accuracy_mean,accuracy_std,recall_mean,precision_mean,f1_mean
2,Naive Bayes,"[0.5874285714285714, 0.5831903945111492, 0.604...",0.583352,0.015004,0.583352,0.340480,0.429938
1,K Nearest Neighbour,"[0.3068571428571429, 0.32075471698113206, 0.60...",0.422263,0.148746,0.422263,0.196007,0.262665
4,Neural Network,"[0.5874285714285714, 0.32075471698113206, 0.29...",0.416514,0.145755,0.416514,0.190479,0.256571
3,Support Vector Machine,"[0.3068571428571429, 0.32075471698113206, 0.29...",0.360399,0.114069,0.360399,0.140297,0.198441
5,Decision Tree,"[0.3068571428571429, 0.32075471698113206, 0.29...",0.360399,0.114069,0.360399,0.140297,0.198441
6,Random Forest,"[0.3068571428571429, 0.32075471698113206, 0.29...",0.360399,0.114069,0.360399,0.140297,0.198441
0,Logistic Regression,"[0.3068571428571429, 0.32075471698113206, 0.29...",0.313058,0.012269,0.313058,0.098126,0.149385
7,Gradient Boosting,"[0.3068571428571429, 0.32075471698113206, 0.29...",0.313058,0.012269,0.313058,0.098126,0.149385
8,AdaBoost,"[0.3068571428571429, 0.32075471698113206, 0.29...",0.313058,0.012269,0.313058,0.098126,0.149385


## Select model with the best performance

In [23]:
tfidf_vect = TfidfVectorizer()
X_train = tfidf_vect.fit_transform(X_train)

model = MultinomialNB()
model.fit(X_train, y_train)

print("Training selesai")

pickle.dump(model, open("model.p", 'wb'))

Training selesai


In [24]:
X_test = tfidf_vect.transform(X_test)

test = model.predict(X_test)
print("Testing selesai")
print(classification_report(y_test, test))

Testing selesai
              precision    recall  f1-score   support

    negative       0.72      0.68      0.70       674
     neutral       0.98      0.19      0.31       232
    positive       0.80      0.95      0.87      1281

    accuracy                           0.78      2187
   macro avg       0.83      0.60      0.63      2187
weighted avg       0.80      0.78      0.76      2187



## Prediksi Data Baru

In [35]:
input_text = "kasur bagus"

# normalize text
text = tfidf_vect.transform([cleansing(input_text)])

# predict data baru menggunakan model yang telah dibuat
result = model.predict(text)[0]

print("Sentiment:", result)

Sentiment: positive


In [26]:
input_text = "kamu hebat sekali"

# normalize text
text = tfidf_vect.transform([cleansing(input_text)])

# predict data baru menggunakan model yang telah dibuat
result = model.predict(text)[0]

print("Sentiment:", result)

Sentiment: positive
